In [1]:
import geopandas as gpd
import pandas as pd
from tqdm import tqdm
import requests
import numpy as np

In [2]:
url = 'https://www.nadlan.gov.il/Nadlan.REST/Main/GetAssestAndDeals'

list_res=[]

In [13]:
dict_data = [{"city_name":  "רעננה","obj_id": "8700","city_name_eng":"raanana","x":196060.80932654, "y":661996.4329939},
 {"city_name":  "הוד השרון","obj_id": "9700","city_name_eng":"hodhasharon","x":196060.80932654, "y":661996.4329939},
 {"city_name":  "רמלה","obj_id": "8500","city_name_eng":"ramla","x":196060.80932654, "y":661996.4329939},
 # {"city_name":  "לוד","obj_id": "8700","city_name_eng":"lod","x":196060.80932654, "y":661996.4329939}, # couldnt find code in nadlan
 {"city_name":  "רחובות","obj_id": "8400","city_name_eng":"rehovot","x":196060.80932654, "y":661996.4329939},
  {"city_name":  "ראשון לציון","obj_id": "8300","city_name_eng":"rishon lezion","x":196060.80932654, "y":661996.4329939},
  {"city_name":   "אלעד","obj_id": "1309","city_name_eng":"elad","x":196060.80932654, "y":661996.4329939},
  {"city_name":  "באר יעקב","obj_id": "2530","city_name_eng":"ber yakov","x":184707.28092654, "y":649639.3578939},
  {"city_name":  "גבעת שמואל","obj_id": "681","city_name_eng":"givat shmuel","x":186077.10822654, "y":664944.1708939},
  {"city_name":  "גני תקווה","obj_id": "229","city_name_eng":"ganei tikva","x":188245.71802654, "y":663207.2540939},
  # {"city_name":  "יבנה","obj_id": "8700","city_name_eng":"yavne"}, # couldnt find code in nadlan
  {"city_name":  "כפר סבא","obj_id": "6900","city_name_eng":"kfar saba","x":196060.80932654, "y":661996.4329939},
  {"city_name":  "כפר קאסם","obj_id": "634","city_name_eng":"kfar qasim","x":196060.80932654, "y":661996.4329939},
# {"city_name":  "טייבה","obj_id": "8700","city_name_eng":"taybe"}, # couldnt find code in nadlan
  {"city_name":  "ראש העין","obj_id": "2640","city_name_eng":"rosh ain","x":196060.80932654, "y":661996.4329939},
  {"city_name":  "קלנסוה","obj_id": "638","city_name_eng":"qalansawe","x":196060.80932654, "y":661996.4329939},
  {"city_name":  "א-טירה","obj_id": "2720","city_name_eng":"tira","x":196060.80932654, "y":661996.4329939},
  {"city_name":  "מודיעין","obj_id": "1200","city_name_eng":"modiin","x":196060.80932654, "y":661996.4329939},
  {"city_name":  "נתניה","obj_id": "7400","city_name_eng":"netania","x":196060.80932654, "y":661996.4329939},
  {"city_name":  "נס ציונה","obj_id": "7200","city_name_eng":"ness ziona","x":196060.80932654, "y":661996.4329939},
{"city_name":  "אשדוד","obj_id": "70","city_name_eng":"ashdod","x":196060.80932654, "y":661996.4329939}
]

In [10]:
import os
os.listdir("../tmp/")

['nadlan_till_55000.csv',
 'nadlan_till_59000.csv',
 '.DS_Store',
 'nadlan_till_108000.csv',
 'nadlan_till_95327.csv',
 'nadlan_till_41000.csv',
 'nadlan_till_69000.csv',
 'nadlan_till_18000.csv',
 'nadlan_till_98000.csv',
 'nadlan_till_0.csv',
 'nadlan_till_37000.csv',
 'nadlan_till_62000.csv',
 'nadlan_till_39000.csv',
 'nadlan_till_56000.csv',
 'nadlan_till_109000.csv',
 'nadlan_till_23000.csv',
 'nadlan_till_99000.csv',
 'nadlan_till_87000.csv',
 'nadlan_till_66000.csv',
 'nadlan_till_50000.csv',
 'nadlan_till_33000.csv']

In [28]:
folder_path = "../nadlan/tmp/"

In [34]:
for n in dict_data[17:]:
    city_name, objid, city_name_eng,X,Y = n.values()
    nav_text = None
    print(city_name, objid, city_name_eng,X,Y)
    df_full=pd.DataFrame()
    list_res=[]
    for start_n in np.arange(1,1801,200):
        if f"{city_name_eng}_nadlan_pages_{start_n+200}.csv" in os.listdir(folder_path):
            continue
        end_n = 200+start_n
        for pagenum in tqdm(range(start_n,end_n)):
            data = {"MoreAssestsType":0,"FillterRoomNum":0,"GridDisplayType":0,
                    "ResultLable":city_name,
                    "ResultType":1,
                    "ObjectID":objid,"ObjectIDType":"text","ObjectKey":"UNIQ_ID",
                    "DescLayerID":"SETL_MID_POINT","Alert":None,
                    "X":X,
                    "Y":Y,
                    "Gush":"","Parcel":"","showLotParcel":True,
                    "showLotAddress":True,"OriginalSearchString":city_name,"MutipuleResults":False,
                    "ResultsOptions":None,
                    "CurrentLavel":2,
                    "Navs":[{"text":nav_text,"url":None,"order":1}],
                    "QueryMapParams":{"QueryToRun":None,"QueryObjectID":objid,"QueryObjectType":"number",
                                      "QueryObjectKey":"SETL_CODE","QueryDescLayerID":"KSHTANN_SETL_AREA","SpacialWhereClause":None},
                    "isHistorical":True,
                    "PageNo":pagenum,
                    "OrderByFilled":"DEALDATETIME","OrderByDescending":True,"Distance":0}
            res = requests.post(url, json=data)
            if res.status_code !=200: 
                break
            result = res.json()
            if  len(result["AllResults"])==0:
                break
            list_res.append(result)
        for r in list_res:
            df= pd.DataFrame(r["AllResults"])
            df_full = pd.concat([df,df_full])
        df_full.to_csv(f"{folder_path}{city_name_eng}_nadlan_pages_{start_n+200}.csv")

אשדוד 70 ashdod 196060.80932654 661996.4329939


100%|█████████████████████████████████████████| 200/200 [22:01<00:00,  6.61s/it]


OSError: [Errno 28] No space left on device